In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset

In [2]:
class BinaryModel(nn.Module):
    def __init__(self, bits=8):
        super().__init__()
        
        self.binary_to_decimal = nn.Linear(in_features=bits, out_features=1, bias=False)
        with torch.no_grad():
            self.binary_to_decimal.weight.copy_(torch.tensor([[2.**i for i in range(bits-1, -1, -1)]], dtype=torch.float32))
        
        for param in self.binary_to_decimal.parameters():
            param.requires_grad = False

    def forward(self, input):
        output = self.binary_to_decimal(input)
        return output


def main():
    bits = 8
    model = BinaryModel(bits=bits)

    input = torch.tensor([[1, 0, 1, 1, 0, 0, 1, 0]], dtype=torch.float32)
    
    output = model(input)
    print(output.item())

if __name__ == '__main__':
    main()

178.0
